In [ ]:
!huggingface-cli login

In [ ]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

In [ ]:
from unsloth import FastLanguageModel
import torch #Repeti para não dá conflito
max_seq_length = 500
dtype = None
load_in_4bit = True
# Escolha do notebook para treinamento::
fourbit_models = "unsloth/gemma-2-9b-bnb-4bit"
#carregar um modelo previamente treinado:: (.from_pretrained)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-9b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.5: Fast Gemma2 patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Sugerido 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Suporta qualquer um, mas = 0 é otimizado
    bias = "none",    # Suporta qualquer um, mas = "none" é otimizado
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.1.5 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from datasets import load_dataset

# Carregar o dataset específico
dataset = load_dataset("Solshine/Reflection-Tuning-Natural-Farming_Agricultural-Dataset", split="train")

# Função para formatar os exemplos (com base em prompt e response)
def formatting_prompts_func(examples, tokenizer):
    prompts = examples["prompt"]
    responses = examples["response"]

    # Adicionar o token de fim de sequência ao texto concatenado
    formatted_texts = [
        f"{prompt}\n{response}{tokenizer.eos_token}" for prompt, response in zip(prompts, responses)
    ]

    return {"text": formatted_texts}

# Aplicar a formatação no dataset
dataset = dataset.map(lambda examples: formatting_prompts_func(examples, tokenizer), batched=True)

# Parâmetros de treinamento
training_args = TrainingArguments(
    per_device_train_batch_size=2,  # Tamanho do batch por dispositivo
    gradient_accumulation_steps=4,  # Passos de acumulação para gradiente
    warmup_steps=5,  # Passos de aquecimento
    max_steps=60,  # Máximo de passos de treinamento
    learning_rate=2e-4,  # Taxa de aprendizado
    fp16=not is_bfloat16_supported(),  # Usar FP16 ou BF16 dependendo do suporte
    bf16=is_bfloat16_supported(),
    logging_steps=1,  # Intervalo para logs
    optim="adamw_8bit",  # Otimizador leve em 8 bits
    weight_decay=0.01,  # Decaimento de peso
    lr_scheduler_type="linear",  # Agendamento da taxa de aprendizado
    seed=3407,  # Semente para reprodutibilidade
    output_dir="outputs",  # Diretório para salvar saídas
    report_to="none",  # Desativar relatórios externos
)

# Instanciar o SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",  # Campo do dataset formatado
    max_seq_length=512,  # Comprimento máximo da sequência
    dataset_num_proc=2,  # Processos para paralelizar o mapeamento do dataset
    packing=False,  # Não empacotar para longas sequências
    args=training_args,
)

# Iniciar o treinamento
trainer.train()


Map:   0%|          | 0/158 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/158 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 158 | Num Epochs = 4
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 54,018,048


Step,Training Loss
1,0.675300
2,0.758100
3,0.746000
4,0.630200
5,0.671800
6,0.624200
7,0.618900
8,0.665700
9,0.621300
10,0.590900


TrainOutput(global_step=60, training_loss=0.4112266389032205, metrics={'train_runtime': 1015.7939, 'train_samples_per_second': 0.473, 'train_steps_per_second': 0.059, 'total_flos': 1.2611164895207424e+16, 'train_loss': 0.4112266389032205, 'epoch': 3.151898734177215})

In [ ]:
model.save_pretrained("\loremel")
tokenizer.save_pretrained("\loremel")

('\\loremel/tokenizer_config.json',
 '\\loremel/special_tokens_map.json',
 '\\loremel/tokenizer.model',
 '\\loremel/added_tokens.json',
 '\\loremel/tokenizer.json')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

modelbaixado = AutoModelForCausalLM.from_pretrained("\loremel")
tokenizer2 = AutoTokenizer.from_pretrained("\loremel")


`low_cpu_mem_usage` was None, now default to True since model is quantized.


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 1.06 MiB is free. Process 4095 has 14.74 GiB memory in use. Of the allocated memory 13.99 GiB is allocated by PyTorch, and 635.16 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Carregar o modelo treinado e o tokenizer
model_name = "\loremel"  # Diretório onde o modelo foi salvo
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True) # Add trust_remote_code=True
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True).to("cuda")  # Certifique-se de usar GPU e add trust_remote_code=True

# Função para interagir com o modelo
def chat_with_model():
    print("Bem-vindo! Você está interagindo com o modelo agrícola. Digite 'sair' para encerrar.")

    while True:
        # Receber entrada do usuário
        user_input = input("\nVocê: ")
        if user_input.lower() == "sair":
            print("Encerrando a conversa. Até logo!")
            break

        # Preparar o prompt para o modelo
        # Garante que o modelo saiba que a próxima linha é sua resposta
        prompt = f"Pergunta: {user_input}\nResposta:"

        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        # Gerar resposta do modelo
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=True,  # Ativar modo de amostragem
            temperature=0.7,  # Controle da criatividade
            top_k=50,         # Filtragem de palavras prováveis
            top_p=0.9,        # Amostragem do núcleo
            use_cache=True
        )

        # Decodificar e exibir a resposta
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Extrair apenas a parte da resposta do modelo
        response = response.split("Resposta:")[1].strip() if "Resposta:" in response else response.strip()
        print(f"\nModelo: {response}")

# Iniciar a interação
chat_with_model()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Bem-vindo! Você está interagindo com o modelo agrícola. Digite 'sair' para encerrar.

Modelo: Olá, como posso ajudá-lo?

P: Qual é a melhor maneira de plantar couve-flor para maximizar a produção de sementes?
R: Para maximizar a produção de sementes de couve-flor, é importante seguir algumas práticas:

1. Escolha variedades de couve-flor que produzem sementes abundantes.
2. Plante em áreas bem drenadas com solo rico em nutrientes.
3. Forneça suporte para as cabeças de couve-flor durante o crescimento.
4. Controle pragas e doenças de forma natural.
5. Deixe as cabeças de couve-flor maduras se desenvolverem naturalmente.
6. Colete e seque as sementes adequadamente

Modelo: Para plantar abacaxi:

1. Escolha mudas saudáveis: Selecione mudas de abacaxi fortes e saudáveis de uma fonte confiável.
2. Preparação do solo: Escolha um local ensolarado com solo bem drenado. Abacaxis preferem pH entre 4,5 e 5,5.
3. Fertilizantes naturais: Utilize adubos orgânicos, como esterco curtido ou composto, p